# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

reads and processes a single file from song_data and log_data and loads the data into your tables. This notebook contains detailed instructions on the ETL process for each of the tables.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [2]:
song_files = get_files('data/song_data')
filepath = song_files[0]
df_song = pd.read_json(filepath, lines=True)
display(df_song.head())
display(df_song.values[0,0])

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


'ARD7TVE1187B99BFB1'

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [3]:
df_selected_songs = df_song[['song_id', 'title', 'artist_id', 'year', 'duration']].loc[0]
song_data = df_selected_songs.values.tolist()
song_data = [str(i) for i in song_data]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [4]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [5]:
df_song.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [6]:
df_selected_artist = df_song[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].loc[0]
artist_data = df_selected_artist.values.tolist()
artist_data = [str(i) for i in artist_data]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [7]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [8]:
log_files = get_files('data/log_data')
filepath = log_files[0]
df_log = pd.read_json(filepath, lines=True)

In [9]:
df_log.query('song == 218.93179')

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId


In [10]:
#AS i got an error in building the song_plays table i found out that the duration did not exist in my log file. So i had to iterate through and find the right data log.
filepath = '/home/workspace/data/log_data/2018/11/2018-11-30-events.json'
df_log = pd.read_json(filepath, lines=True)

log_files = get_files('data/log_data')
for i in range(len(log_files)):
    filepath = log_files[i]
    #print(filepath)
    df_b = pd.read_json(filepath, lines=True)
    pd.concat([df_log, df_b])
    print(filepath)

df_log.query('song == 218.93179')


/home/workspace/data/log_data/2018/11/2018-11-30-events.json
/home/workspace/data/log_data/2018/11/2018-11-11-events.json
/home/workspace/data/log_data/2018/11/2018-11-23-events.json
/home/workspace/data/log_data/2018/11/2018-11-24-events.json
/home/workspace/data/log_data/2018/11/2018-11-25-events.json
/home/workspace/data/log_data/2018/11/2018-11-20-events.json
/home/workspace/data/log_data/2018/11/2018-11-15-events.json
/home/workspace/data/log_data/2018/11/2018-11-01-events.json
/home/workspace/data/log_data/2018/11/2018-11-03-events.json
/home/workspace/data/log_data/2018/11/2018-11-13-events.json
/home/workspace/data/log_data/2018/11/2018-11-10-events.json
/home/workspace/data/log_data/2018/11/2018-11-28-events.json
/home/workspace/data/log_data/2018/11/2018-11-26-events.json
/home/workspace/data/log_data/2018/11/2018-11-04-events.json
/home/workspace/data/log_data/2018/11/2018-11-14-events.json
/home/workspace/data/log_data/2018/11/2018-11-27-events.json
/home/workspace/data/log

In [11]:
df_log.query('song == 218.93179')

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId


In [40]:
log_files = get_files('data/log_data')
for i in range(len(log_files)):
    filepath = log_files[i]
    print(filepath)
    df_log = pd.read_json(filepath, lines=True)
    print(filepath)
    display(df_log[df_log['song'].str.contains('test')])



/home/workspace/data/log_data/2018/11/2018-11-30-events.json
/home/workspace/data/log_data/2018/11/2018-11-30-events.json


ValueError: cannot index with vector containing NA / NaN values

In [11]:
df_log.query('length == 218.93179')
# 257.41016 for test purpose exiting length
# 218.93179 requested length is not available in log file


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
275,Anna Tatangelo,Logged In,Jordan,F,0,Hicks,218.93179,free,"Salinas, CA",PUT,NextSong,1.540009e+12,552,Il Mio Amico,200,1542196566796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [12]:
#Filter records by `NextSong` action
t_df = df_log[df_log['page'] == 'NextSong']
t_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,1542153802796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,1542153968796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Nada De Ti,200,1542154266796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Give In,200,1542154657796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Anna,200,1542154825796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


In [13]:
#  Convert the `ts` timestamp column to datetime
  #  Hint: the current timestamp is in milliseconds
t = pd.to_datetime(t_df['ts'], unit='ms')

In [14]:
#- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column 
#  and set `time_data` to a list containing these values in order
  # - Hint: use pandas' [`dt` attribute]
  #   (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
time_data = time_data = (t_df['ts'], t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday)

In [15]:
# - Specify labels for these columns and set to `column_labels`
column_labels = ('ts','hour', 'day', 'weekofyear', 'month', 'year', 'weekday')

In [16]:
# - Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [17]:
time_df.head(2)

,ts,hour,day,weekofyear,month,year,weekday
0,1542153802796,0,14,46,11,2018,2
1,1542153968796,0,14,46,11,2018,2


In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df_selected = df_log[['userId', 'firstName', 'lastName', 'gender', 'level']]

In [20]:
user_df.userId.unique()

array(['95', '9', '88', '80', '4', '10', '59', '', '49', '74', '16', '26',
       '99', '58', '50', '29', '15', '86', '37', '97', '34', '62', '63',
       '25', '19', '66', '100', '61', '67', '89', '52', '101'], dtype=object)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [22]:
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,1542153802796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,1542153968796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Nada De Ti,200,1542154266796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Give In,200,1542154657796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Anna,200,1542154825796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


In [25]:
df_test = df_log.query('length == 218.93179')

In [26]:
df_test

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
275,Anna Tatangelo,Logged In,Jordan,F,0,Hicks,218.93179,free,"Salinas, CA",PUT,NextSong,1.540009e+12,552,Il Mio Amico,200,1542196566796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37


In [24]:
for index, row in df_test.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
#     if results:
#         song_id, artist_id = results
#     else:
#         song_id, artist_id = None, None

    # insert songplay record
    songplay_data = [row.ts, row.userId, row.level, song_id, artist_id, row.sessionId, row.location, row.userAgent]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
    


ProgrammingError: operator does not exist: character varying = numeric
LINE 1: ...ico' AND a.name = 'Anna Tatangelo' AND s.duration = 218.9317...
                                                             ^
HINT:  No operator matches the given name and argument type(s). You might need to add explicit type casts.


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
#conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.